# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234682335898                   -0.50   10.0
  2   -7.250339586969       -1.81       -1.40    1.0
  3   -7.251125623644       -3.10       -2.05    1.0
  4   -7.251162075907       -4.44       -2.00    2.0
  5   -7.251329930888       -3.78       -2.62    1.0
  6   -7.251338179003       -5.08       -3.14    1.0
  7   -7.251338724922       -6.26       -3.63    2.0
  8   -7.251338791123       -7.18       -4.02    2.0
  9   -7.251338797444       -8.20       -4.52    1.0
 10   -7.251338798564       -8.95       -5.01    2.0
 11   -7.251338798688       -9.91       -5.29    3.0
 12   -7.251338798697      -11.03       -5.55    1.0
 13   -7.251338798704      -11.14       -6.39    1.0
 14   -7.251338798704      -12.40       -6.65    2.0
 15   -7.251338798705      -13.24       -7.06    1.0
 16   -7.251338798705      -14.15       -7.37    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.052784354460070064
[ Info: Arnoldi iteration step 2: normres = 0.4357399772889116
[ Info: Arnoldi iteration step 3: normres = 1.00536250060753
[ Info: Arnoldi iteration step 4: normres = 0.34057425480778314
[ Info: Arnoldi iteration step 5: normres = 0.5183687282812663
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.67e-02, 7.18e-02, 4.61e-01, 2.16e-01, 7.85e-03)
[ Info: Arnoldi iteration step 6: normres = 0.3026826061259115
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.63e-02, 1.01e-01, 2.34e-01, 1.29e-01, 8.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08260029027757071
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.51e-04, 8.83e-03, 1.63e-02, 4.66e-02, 5.63e-02)
[ Info: Arnoldi iteration step 8: normres = 0.0985175863496371
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.76e-05